# HiCoTiNe 2. Adapted from HiCoTiNe v3.3.1
## Concatenated hidden representations.
## No raw input
## The weight of each TN is inversely proportional to the number of TNs in its HiCo layer, relative to the HiCo layer with the most TNs

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K

In [2]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [10]:
NUM_BATCH = 28
BATCH_SIZE = 128
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_TEST_STEP = 10
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]
ALPHA = 0.1

#dataset specific parameters
NUM_CLASS = 5

In [11]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [12]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [13]:
train = np.array(train_images)
train = train.astype('float32')
train.shape

(4323, 32, 32, 3)

In [14]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,train_labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

## Data Pre-Processing

In [15]:
#Generate cropped train image
X_train_cropped_list = []
y_train_cropped_list = []
#coordinate_list = []
for i in range(NUM_TN[0]):
  X_train_cropped_batch = []
  y_train_cropped_batch = []
  for j in range(NUM_BATCH):
    X_train_cropped = []
    y_train_cropped = []
    sub_region_scale = rn.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
    x = rn.randint(0, X_train.shape[1] - sub_region_scale)
    y = rn.randint(0, X_train.shape[1] - sub_region_scale)
    
    if (j == NUM_BATCH - 1) and ((len(X_train) - j*BATCH_SIZE) % BATCH_SIZE):
      for k in range(len(X_train) - j*BATCH_SIZE):
        image_cropped = []
        for row in range(y, y + sub_region_scale):
          image_cropped_row = []
          for column in range(x, x + sub_region_scale):
            image_cropped_column = []
            for rgb in range(3):
              image_cropped_column.append(X_train[(j*BATCH_SIZE)+k][row][column][rgb])
            image_cropped_row.append(image_cropped_column)
          image_cropped.append(image_cropped_row)
        X_train_cropped.append(image_cropped)
        y_train_cropped.append(y_train[(j*BATCH_SIZE)+k])
      
      X_train_cropped = np.array(X_train_cropped)
      y_train_cropped = np.array(y_train_cropped)
      X_train_cropped_batch.append(X_train_cropped)
      y_train_cropped_batch.append(y_train_cropped)
      break

    for k in range(BATCH_SIZE):
      image_cropped = []
      for row in range(y, y + sub_region_scale):
        image_cropped_row = []
        for column in range(x, x + sub_region_scale):
          image_cropped_column = []
          for rgb in range(3):
            image_cropped_column.append(X_train[(j*BATCH_SIZE)+k][row][column][rgb])
          image_cropped_row.append(image_cropped_column)
        image_cropped.append(image_cropped_row)
      X_train_cropped.append(image_cropped)
      y_train_cropped.append(y_train[(j*BATCH_SIZE)+k])

    X_train_cropped = np.array(X_train_cropped)
    y_train_cropped = np.array(y_train_cropped)
    X_train_cropped_batch.append(X_train_cropped)
    y_train_cropped_batch.append(y_train_cropped)
  X_train_cropped_list.append(X_train_cropped_batch)
  y_train_cropped_list.append(y_train_cropped_batch)

In [16]:
#Generate cropped val image
X_val_cropped_list = []
for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_val_cropped = []
  sub_region_scale = rn.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = rn.randint(0, X_val.shape[1] - sub_region_scale)
  y = rn.randint(0, X_val.shape[1] - sub_region_scale)
  for j in range(X_val.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_val[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_val_cropped.append(image_cropped)

  X_val_cropped = np.array(X_val_cropped)
  X_val_cropped_list.append(X_val_cropped)

In [17]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped = []
  sub_region_scale = rn.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = rn.randint(0, X_test.shape[1] - sub_region_scale)
  y = rn.randint(0, X_test.shape[1] - sub_region_scale)
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_column = []
        for rgb in range(3):
          image_cropped_column.append(X_test[j][row][column][rgb])
        image_cropped_row.append(image_cropped_column)
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [18]:
#rescaling dataset
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    temp = []
    for k in range (len(X_train_cropped_list[i][j])):
      temp.append(cv2.resize(X_train_cropped_list[i][j][k], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
    temp = np.array(temp)
    X_train_cropped_list[i][j] = temp

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [19]:
for i in range(NUM_TN[0] * NUM_TEST_STEP):
  temp = []
  for j in range(X_val.shape[0]):
    temp.append(cv2.resize(X_val_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_val_cropped_list[i] = temp

In [20]:
#normalizing dataset
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    X_train_cropped_list[i][j] = X_train_cropped_list[i][j]/255

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_val_cropped_list[i] = X_val_cropped_list[i]/255

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [21]:
#transform to vector
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    X_train_cropped_list[i][j] = X_train_cropped_list[i][j].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_val_cropped_list[i] = X_val_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [22]:
#one-hot encoding
y_train_one_hot = []
for i in range(NUM_TN[0]):
  temp = []
  for j in range(NUM_BATCH):
    y_train_cropped_list[i][j] = pd.get_dummies(y_train_cropped_list[i][j])
    y_train_cropped_list[i][j] = y_train_cropped_list[i][j].values.argmax(1)
    temp.append(to_categorical(y_train_cropped_list[i][j], num_classes=NUM_CLASS))
  y_train_one_hot.append(temp)


y_val_dummed = pd.get_dummies(y_val)
y_val_dummed = y_val_dummed.values.argmax(1)
y_val_one_hot = to_categorical(y_val_dummed, num_classes=NUM_CLASS)

y_test_dummed = pd.get_dummies(y_test)
y_test_dummed = y_test_dummed.values.argmax(1)
y_test_one_hot = to_categorical(y_test_dummed, num_classes=NUM_CLASS)

## HiCo Layer 1

In [23]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i))

In [24]:
for i in range(NUM_TN[0]):
  print('Model %d' %i)
  for j in range(5):
    print('Epoch %d' %j)
    for k in range(NUM_BATCH):
      ensemble[i][0].train_on_batch(X_train_cropped_list[i][k], y_train_one_hot[i][k])

Model 0
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 1
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 2
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 3
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 4
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 5
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 6
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 7
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 8
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 9
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [25]:
hico_layers = []
hico_layers.append(ensemble)

In [26]:
del hico_layers[1:]

## HiCo Layer 2+

In [27]:
X_train_cropped_list_combined = []
for i in range(NUM_TN[0]):
  X_train_cropped_list_combined.append(np.concatenate(X_train_cropped_list[i], axis=0))

y_train_dummed = pd.get_dummies(y_train)
y_train_dummed = y_train_dummed.values.argmax(1)
y_train_one_hot_combined = to_categorical(y_train_dummed, num_classes=NUM_CLASS)


In [28]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.concatenate(input, axis=1)
    previous_layer_input = input
    return previous_layer_input

In [30]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(rn.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list_combined)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.concatenate(X_train_input, axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot_combined, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5)

Layer 1
HICO LENGTH
3
Model 0
Epoch 1/5
109/109 [==============================] - 1s 4ms/step - loss: 1.4314 - accuracy: 0.3460 - val_loss: 1.3676 - val_accuracy: 0.3857
Epoch 2/5
109/109 [==============================] - 0s 3ms/step - loss: 1.3055 - accuracy: 0.4275 - val_loss: 1.3465 - val_accuracy: 0.4249
Epoch 3/5
109/109 [==============================] - 0s 3ms/step - loss: 1.2579 - accuracy: 0.4722 - val_loss: 1.3562 - val_accuracy: 0.3926
Epoch 4/5
109/109 [==============================] - 0s 2ms/step - loss: 1.2606 - accuracy: 0.4588 - val_loss: 1.3569 - val_accuracy: 0.3949
Epoch 5/5
109/109 [==============================] - 0s 2ms/step - loss: 1.2217 - accuracy: 0.4683 - val_loss: 1.3352 - val_accuracy: 0.4065
Model 1
Epoch 1/5
109/109 [==============================] - 1s 4ms/step - loss: 1.4293 - accuracy: 0.3503 - val_loss: 1.3788 - val_accuracy: 0.3695
Epoch 2/5
109/109 [==============================] - 0s 2ms/step - loss: 1.3052 - accuracy: 0.4239 - val_loss: 1.346

IndexError: list index out of range

## Model Evaluation

In [31]:
# Predict on test image
y_pred_list_2 = []
for h in range(NUM_TEST_STEP):
  y_pred_list = []
  for j in range(NUM_TN[0]):
    y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[(h*NUM_TN[0])+j])
    y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[0] / max(NUM_TN)))
    y_pred_list.append(y_pred)

  for i in range(1, NUM_HICO_LAYER):
    for j in range(NUM_TN[i]):

      # Get test hidden representation from HiCo layer i-1
      X_test_input = []
      for k in range(NUM_CONNECTION[i]):
        get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
        X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list[h*NUM_TN[0]:(h+1)*NUM_TN[0]])))
      X_test_input = np.array(X_test_input)
      X_test_input = np.concatenate(X_test_input, axis=1)
      y_pred = hico_layers[i][j][0].predict(X_test_input)
      y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[i] / max(NUM_TN)))
      y_pred_list.append(y_pred)

  # HiCo voting
  y_pred_list = np.array(y_pred_list)
  y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
  y_pred_list = np.sum(y_pred_list, axis=1)
  y_pred_list_2.append(y_pred_list)

y_pred_list_2 = np.array(y_pred_list_2)
y_pred_list_2 = np.transpose(y_pred_list_2, (1, 0, 2))
y_pred_list_2 = np.argmax(np.sum(y_pred_list_2, axis=1), axis=1)

IndexError: list index out of range

In [ ]:
y_test_label_encoded_list =[]

In [ ]:
for k in y_test_cropped_list:
    y_test_label_encoded_list.append(k)
    
y_test_label_encoded_list  = np.array(y_test_label_encoded_list)
y_test_label_encoded_list = np.argmax(np.sum(y_test_label_encoded_list, axis=0), axis=1)

In [ ]:
accuracy = accuracy_score(y_test_label_encoded_list, y_pred_list)
accuracy

In [ ]:
accuracy = accuracy_score(y_test, y_pred_list_2)
accuracy